In [1]:
from tensorflow import keras
import os
# from helper import utils
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from IPython.display import Audio 
import torch
import librosa
import numpy as np
import soundfile as sf
from scipy.io import wavfile
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer,pipeline

2023-06-15 01:33:18.084975: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-15 01:33:18.258194: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-15 01:33:18.259704: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-15 01:33:19.514820: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/pranav/Documents/Python/Project/Project Parts/temp/lib64/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class utils:
    choosemicdevice=True
    dev=0
    userid=0
    label=0
    cmd=0
    
    def extract_features(self,files):
        import os
        import librosa
        import numpy as np
        # Sets the name to be the path to where the file is in my computer
        file_name = os.path.join(os.path.abspath(self.userid+'/'+str(files.file)))
        # Loads the audio file as a floating point time series and assigns the default sample rate
        # Sample rate is set to 22050 by default
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        # Generate Mel-frequency cepstral coefficients (MFCCs) from a time series 
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
        # Generates a Short-time Fourier transform (STFT) to use in the chroma_stft
        stft = np.abs(librosa.stft(X))
        # Computes a chromagram from a waveform or power spectrogram.
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        # Computes a mel-scaled spectrogram.
        mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
        # Computes spectral contrast
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
        # Computes the tonal centroid features (tonnetz)
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
        sr=sample_rate).T,axis=0)
        # We add also the classes of each file as a label at the end
        label = files.label
        return mfccs, chroma, mel, contrast, tonnetz,[label]
    
    def recordAudio(self,filename,duration):
        import sounddevice as sd
        from scipy.io.wavfile import write

        fs = 48000  # Sample rate
        seconds = duration # Duration of recording
        if self.choosemicdevice==True:
            print(":---Input Devices available---:")
            print(sd.query_devices())
            self.dev=int(input("Select the device:"))
            print(":---Input Device Selected---:")
            sd.default.device=sd.query_devices()[self.dev]['name']
            print(sd.default.device)
            self.choosemicdevice=False
        print("Recording")
        myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2)
        sd.wait()  # Wait until recording is finished
        print(f"Saving it as {filename}")
        write(filename, fs, myrecording)
    
    def getVoiceSamples(self,duration):
        import os
        import librosa
        import numpy as np
        import uuid
        text_lines = {
        "Hello": 2.5,
        "Can you please repeat": 4.2,
        "What is your favorite color": 5.1,
        "Could you introduce yourself": 6.3,
        "Please describe your most memorable vacation": 7.8,
        "Would you mind reading this passage": 9.2,
        "Can you sing a few lines from your favorite song": 11.5,
        "Tell me a joke that always makes you laugh": 4.7,
        "What is your opinion on artificial intelligence": 6.8,
        "Could you recite a famous quote or poem": 8.1
        }
        print('Repeat the lines given below to generate your voice sample')
        speakerid=uuid.uuid1().hex[:10]
        lineCount=0
        os.mkdir(speakerid)
        for line in text_lines:
            print("---"+line+"---")
            lineCount+=1
            utils.recordAudio(self,speakerid+"/"+str(lineCount)+'.flac',duration)
            self.userid=speakerid
#             if(lineCount==5):break
        self.label=input("Please Specify your gender \n 0:Female\n1:Male")
        return speakerid
    
    def takeCommand(self,duration):
        self.cmd+=1
        utils.recordAudio(self,self.userid+"/"+"cmd-"+str(self.cmd)+'.flac',duration)
        return "cmd-"+str(self.cmd)+'.flac'
        
        


In [3]:
util= utils()
util.label=1
util.cmd=0
util.userid='642b'

In [414]:
util.choosemicdevice=True
util.recordAudio('testfile1.wav',3)

:---Input Devices available---:
   0 HD-Audio Generic: ALC245 Analog (hw:0,0), ALSA (2 in, 2 out)
   1 acp: - (hw:1,0), ALSA (2 in, 0 out)
   2 sysdefault, ALSA (128 in, 128 out)
   3 front, ALSA (0 in, 2 out)
   4 surround40, ALSA (0 in, 2 out)
   5 surround51, ALSA (0 in, 2 out)
   6 surround71, ALSA (0 in, 2 out)
   7 pipewire, ALSA (64 in, 64 out)
   8 dmix, ALSA (0 in, 2 out)
*  9 default, ALSA (64 in, 64 out)
  10 Family 17h/19h HD Audio Controller Headphones Stereo Microphone, JACK Audio Connection Kit (2 in, 0 out)
  11 Family 17h/19h HD Audio Controller Digital Microphone, JACK Audio Connection Kit (2 in, 0 out)
  12 Family 17h/19h HD Audio Controller Speaker + Headphones, JACK Audio Connection Kit (2 in, 2 out)


KeyboardInterrupt: Interrupted by user

 ## Use util.choosemicdevice=True to again get the device select menu 

In [9]:

Audio('testfile1.wav')

In [10]:
userid=util.getVoiceSamples(4)

Repeat the lines given below to generate your voice sample
---Hello---
Recording
Saving it as fcd4dd5e03/1.wav
---Can you please repeat---
Recording
Saving it as fcd4dd5e03/2.wav
---What is your favorite color---
Recording
Saving it as fcd4dd5e03/3.wav
---Could you introduce yourself---
Recording
Saving it as fcd4dd5e03/4.wav
---Please describe your most memorable vacation---
Recording
Saving it as fcd4dd5e03/5.wav
---Would you mind reading this passage---
Recording
Saving it as fcd4dd5e03/6.wav
---Can you sing a few lines from your favorite song---
Recording
Saving it as fcd4dd5e03/7.wav
---Tell me a joke that always makes you laugh---
Recording
Saving it as fcd4dd5e03/8.wav
---What is your opinion on artificial intelligence---
Recording
Saving it as fcd4dd5e03/9.wav
---Could you recite a famous quote or poem---
Recording
Saving it as fcd4dd5e03/10.wav
Please Specify your gender 
 0:Female
1:Male1


In [4]:
files=os.listdir(util.userid+"/")
data=pd.DataFrame(files)
data=data.rename(columns={0:'file'})

In [5]:
files

['642b-2.wav',
 '642b-cmd.wav',
 '642b-3.wav',
 '642b-1.wav',
 '642b-4.wav',
 '642b-7.wav',
 '642b-5.wav',
 '642b-6.wav',
 '642b-8.wav',
 '642b-10.wav',
 '642b-9.wav']

In [6]:
data['label']=util.label

In [7]:
data

,file,label
0,642b-2.wav,1
1,642b-cmd.wav,1
2,642b-3.wav,1
3,642b-1.wav,1
4,642b-4.wav,1
5,642b-7.wav,1
6,642b-5.wav,1
7,642b-6.wav,1
8,642b-8.wav,1
9,642b-10.wav,1


In [8]:
util.userid

'642b'

In [9]:
features_label = data.apply(util.extract_features, axis=1)

/home/pranav/Documents/Python/Project/Project Parts/temp/lib64/python3.8/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=1024 is too large for input signal of length=1007
  warnings.warn(
/home/pranav/Documents/Python/Project/Project Parts/temp/lib64/python3.8/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=1024 is too large for input signal of length=728
  warnings.warn(
/home/pranav/Documents/Python/Project/Project Parts/temp/lib64/python3.8/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=1024 is too large for input signal of length=807
  warnings.warn(
/home/pranav/Documents/Python/Project/Project Parts/temp/lib64/python3.8/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=1024 is too large for input signal of length=880
  warnings.warn(
/home/pranav/Documents/Python/Project/Project Parts/temp/lib64/python3.8/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=1024 is too large for input signal of length=926
  warn

In [10]:
features_label[0]

(array([-3.4050790e+02,  9.4534790e+01, -2.2407896e+01,  2.0276489e+01,
         6.1983705e+00,  8.4726229e+00,  1.0924594e+00,  8.8670120e+00,
        -1.0177713e+01,  1.6696237e+00, -1.1696605e+01, -2.1293404e+00,
         5.4017764e-01, -6.7449999e-01,  1.4321971e+00,  9.1071177e-01,
        -2.3172741e+00, -7.4099588e-01, -3.9222810e+00, -4.3208737e+00,
        -5.4972849e+00, -2.8950632e+00, -3.7381449e+00, -1.7265031e+00,
         4.9443364e-01,  1.7584762e-01, -7.8356236e-01, -1.1259761e+00,
        -1.2091227e+00, -4.4350874e-01, -2.1978664e+00, -1.9387655e+00,
         2.4875171e-01, -6.4260893e-02, -8.1702435e-01, -1.9600085e+00,
        -1.5442709e+00, -1.2532344e+00, -2.3245723e+00, -1.5253328e+00],
       dtype=float32),
 array([0.7660642 , 0.75767493, 0.7253705 , 0.6826213 , 0.64319   ,
        0.5891762 , 0.67016613, 0.7185347 , 0.71836543, 0.7549484 ,
        0.7855558 , 0.8021017 ], dtype=float32),
 array([5.55903792e-01, 4.71259737e+00, 9.76660442e+00, 6.73391867e+00,

In [19]:
features = []
for i in range(0, len(features_label)):
    features.append(np.concatenate((features_label[i][0], features_label[i][1], 
                features_label[i][2], features_label[i][3],
                features_label[i][4]), axis=0))

In [20]:
X = np.array(features)
from  sklearn.preprocessing import StandardScaler
ss=StandardScaler()
ss.fit(X)
X_train=ss.transform(X)


from joblib import dump, load
ss=load('std_scaler.bin')

X_test = ss.transform(X)

In [21]:
 model = keras.models.load_model("spktvrfi")

In [22]:
predict_x=model.predict(X_train) 
np.max(predict_x,axis=1)


1/1 [==============================] - 0s 67ms/step


array([0.49251664, 0.99999994, 0.9823835 , 0.98494095, 0.9790012 ,
       0.9550725 , 0.88133526, 0.99469715, 0.92930615, 0.8739254 ,
       0.82128274], dtype=float32)

In [23]:
    preds=np.argmax(predict_x,axis=1)

In [24]:
preds

array([204, 196,  56,  52,  82, 129,  29, 239, 189, 153,  10])

In [25]:
authoriseduserid=np.bincount(preds).argmax()

In [26]:
authoriseduserid

10

In [334]:
from keras.utils.np_utils import to_categorical
y_train=[]
for i in range(0,7):
    y_train.append(authoriseduserid)
y_train=np.array(y_train)
lb=load('labelEncoader.bin')
y_train=to_categorical(y_train,num_classes=251)
X_train=X[:7]

In [335]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [336]:
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=1, mode='auto')

history=model.fit(X_train,y_train, batch_size=4, epochs=20,validation_data=(X_train,y_train),callbacks=[early_stop]) 


Epoch 1/20
2/2 [==============================] - 1s 112ms/step - loss: 181.1547 - accuracy: 0.1429 - val_loss: 6.2562 - val_accuracy: 0.8571
Epoch 2/20
2/2 [==============================] - 0s 22ms/step - loss: 7.8381 - accuracy: 0.7143 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/20
2/2 [==============================] - 0s 23ms/step - loss: 1.4932 - accuracy: 0.8571 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/20
2/2 [==============================] - 0s 23ms/step - loss: 32.7229 - accuracy: 0.8571 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/20
2/2 [==============================] - 0s 23ms/step - loss: 25.8014 - accuracy: 0.5714 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/20
2/2 [==============================] - 0s 22ms/step - loss: 21.1911 - accuracy: 0.7143 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/20
2/2 [==============================] - 0s 23ms/step - loss: 9.7156 - accuracy: 0.7143 - val_loss: 0.0000e+00 - val_accuracy: 1.00

In [40]:
type(authoriseduserid.item())

int

In [26]:
filename=util.takeCommand(4)

Recording
Saving it as fcd4dd5e03/cmd-1.flac


In [113]:
filename='cmd-1.flac'

In [27]:
test=[filename]
test=pd.DataFrame(test)
test=test.rename(columns={0:'file'})
test['label']=util.label
features_label = test.apply(util.extract_features, axis=1)
features = []
for i in range(0, len(features_label)):
    features.append(np.concatenate((features_label[i][0], features_label[i][1], 
                features_label[i][2], features_label[i][3],
                features_label[i][4]), axis=0))
X = np.array(features)
# X_test = ss.transform(X)
predict_x=model.predict(X) 
preds=np.argmax(predict_x,axis=1)

1/1 [==============================] - 0s 36ms/step


In [28]:
if preds[0]==authoriseduserid:
    print('Access Granted')
else:
    print('No Access')

Access Granted


In [29]:
preds

array([1])

In [12]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/home/pranav/Documents/Python/Project/Project Parts/temp/lib64/python3.8/site-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:757: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
def stt(filename):
    os.system(f'ffmpeg -i {filename} {filename[:-4]}.wav')
    filename=filename[:-4]+".wav"
    print(filename)
    data = wavfile.read(filename)
    framerate = data[0]
    sounddata = data[1]
    time = np.arange(0,len(sounddata))/framerate
    input_audio, _ = librosa.load(filename, sr=16000)
    input_values = tokenizer(input_audio, return_tensors="pt").input_values
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids)[0]
    return transcription

In [34]:
text=[]
for file in files:
    a=util.userid+"/"+file
    text.append(stt(a))
    

fcd4dd5e03/2.wav


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (GCC)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --docdir=/usr/share/doc/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -flto=auto -ffat-lto-objects -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wp,-U_FORTIFY_SOURCE,-D_FORTIFY_SOURCE=3 -Wp,-D_GLIBCXX_ASSERTIONS -specs=/usr/lib/rpm/redhat/redhat-hardened-cc1 -fstack-protector-strong -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -m64 -mtune=generic -fasynchronous-unwind-tables -fstack-clash-protection -fcf-protection -fno-omit-frame-pointer -mno-omit-leaf-frame-pointer' --extra-ldflags='-Wl,-z,relro -Wl,--as-needed -Wl,-z,now -specs=/usr/lib/rpm/redhat/redhat-hardened-ld -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -Wl,--build-id=sha1 ' --extra-cflags=' -I/usr/include/rav1e' --enable-libopencore-amrnb --enable-libopencore-amrwb -

FileNotFoundError: [Errno 2] No such file or directory: 'fcd4dd5e03/2.wav'

In [35]:
files


['2.wav',
 '1.wav',
 '9.wav',
 '6.wav',
 '10.wav',
 '8.wav',
 '7.wav',
 '5.wav',
 '4.wav',
 '3.wav']

In [44]:
pipe = pipeline(model="facebook/bart-large-mnli")

In [45]:
pipe("send a email to zxy",
    candidate_labels=['weather','email','translate']
    )

{'sequence': 'send a email to zxy',
 'labels': ['email', 'translate', 'weather'],
 'scores': [0.9124555587768555, 0.08164118975400925, 0.005903288256376982]}

In [47]:
for t in text:
    print(pipe(t,candidate_labels=['weather','email','translate']))

{'sequence': 'CAN YOU PLEASE IT A GOOD', 'labels': ['translate', 'email', 'weather'], 'scores': [0.6533355712890625, 0.2850351631641388, 0.0616292804479599]}
{'sequence': 'WILL YOU DISCABE ME MOST MENABEVICATION', 'labels': ['translate', 'email', 'weather'], 'scores': [0.7321274280548096, 0.24423933029174805, 0.023633288219571114]}
{'sequence': 'WHORK IS YO OPING AN UNARTIFICIAL INTULLIGENCE', 'labels': ['translate', 'weather', 'email'], 'scores': [0.6417428851127625, 0.2999660074710846, 0.058291107416152954]}
{'sequence': 'CAN YOU SINGLE FEW LINES FOT ON YOUR FILDERD SONG', 'labels': ['translate', 'email', 'weather'], 'scores': [0.8844776749610901, 0.06496500968933105, 0.05055736377835274]}
{'sequence': 'ALMOST SAID BAN COULD HE SAR TI FER', 'labels': ['translate', 'email', 'weather'], 'scores': [0.8483059406280518, 0.09666095674037933, 0.05503316596150398]}
{'sequence': 'WOULD YOU WY READING THIS POSSAGE', 'labels': ['translate', 'email', 'weather'], 'scores': [0.7377259135246277, 0.

In [341]:
import os

def rename_files(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            folder_name = os.path.basename(root)
            file_name, file_ext = os.path.splitext(file)
            new_name = folder_name + "-" + file_name + file_ext

            # Construct the new file path
            current_path = os.path.join(root, file)
            new_path = os.path.join(root, new_name)

            # Rename the file
            os.rename(current_path, new_path)

# Provide the folder path where the files need to be renamed
folder_path = "8c1d"

# Call the rename_files function with the folder path
rename_files(folder_path)


In [347]:
files=os.listdir("voiceSample/")
data=pd.DataFrame(files)
data=data.rename(columns={0:'file'})
data['label']=util.label

In [348]:
data

,file,label
0,2.wav,1
1,642b-2.wav,1
2,1.wav,1
3,65c8-4.wav,1
4,65c8-6.wav,1
5,65c8-7.wav,1
6,9.wav,1
7,65c8-1.wav,1
8,65c8-5.wav,1
9,642b-cmd.wav,1


In [349]:
def extract_features(files):
    
    # Sets the name to be the path to where the file is in my computer
    file_name = os.path.join(os.path.abspath('voiceSample')+'/'+str(files.file))

    # Loads the audio file as a floating point time series and assigns the default sample rate
    # Sample rate is set to 22050 by default
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 

    # Generate Mel-frequency cepstral coefficients (MFCCs) from a time series 
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)

    # Generates a Short-time Fourier transform (STFT) to use in the chroma_stft
    stft = np.abs(librosa.stft(X))

    # Computes a chromagram from a waveform or power spectrogram.
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)

    # Computes a mel-scaled spectrogram.
    mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)

    # Computes spectral contrast
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)

    # Computes the tonal centroid features (tonnetz)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
        
    
    # We add also the classes of each file as a label at the end
    label = files.label

    return mfccs, chroma, mel, contrast, tonnetz, [label]

In [350]:
features_label = data.apply(extract_features, axis=1)

/home/pranav/Documents/Python/Project/Project Parts/temp/lib64/python3.8/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=1024 is too large for input signal of length=537
  warnings.warn(
/home/pranav/Documents/Python/Project/Project Parts/temp/lib64/python3.8/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=1024 is too large for input signal of length=1007
  warnings.warn(
/home/pranav/Documents/Python/Project/Project Parts/temp/lib64/python3.8/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=1024 is too large for input signal of length=912
  warnings.warn(
/home/pranav/Documents/Python/Project/Project Parts/temp/lib64/python3.8/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=1024 is too large for input signal of length=456
  warnings.warn(
/home/pranav/Documents/Python/Project/Project Parts/temp/lib64/python3.8/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=1024 is too large for input signal of length=552
  warn

In [351]:
features = []
for i in range(0, len(features_label)):
    features.append(np.concatenate((features_label[i][0], features_label[i][1], 
                features_label[i][2], features_label[i][3],
                features_label[i][4],features_label[i][5]), axis=0))

In [354]:
speaker = []
for i in range(0, len(data)):
    speaker.append(data['file'][i].split('-')[0])

In [356]:
data['speaker'] = speaker

In [359]:
X = np.array(features)
y = np.array(speaker)

In [361]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
y = to_categorical(lb.fit_transform(y))

In [362]:
size=data.size
trainSize=int(0.7*size)
validationSize=int(0.2*size)
testSize=int(0.1*size)
X_train = X[:trainSize]
y_train = y[:trainSize]

X_val = X[trainSize:trainSize+validationSize]
y_val = y[trainSize:trainSize+validationSize]

X_test = X[trainSize+validationSize:]
y_test = y[trainSize+validationSize:]


In [365]:
from sklearn.preprocessing import StandardScaler
sss = StandardScaler()
X_train = sss.fit_transform(X_train)
X_val = sss.transform(X_val)
X_test = sss.transform(X_test)

ValueError: Found array with 0 sample(s) (shape=(0, 194)) while a minimum of 1 is required by StandardScaler.